In [ ]:
import io, os, sys, ssl, time, json, socket, struct, random, datetime, threading
import serial, pigpio, picamera
from adafruit_motorkit import MotorKit
from IPython.display import clear_output

server_ip = 'guoxiaokang.net'
img_port = 54321
ctrl_port = 54322 
sensor_port = 54324

server_cert_self_signed  = 'secret/server.crt' 
client_certs_self_signed = 'secret/rover.crt'
client_key_self_signed   = 'secret/rover.key'

# Establish connection with server
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert_self_signed)
context.load_cert_chain(certfile=client_certs_self_signed, keyfile=client_key_self_signed)

In [ ]:
#kit = MotorKit() 
pi = pigpio.pi(port=8889) ### Start PiGPIO Client
rotation_pin, elevation_pin = 17, 27  # {sky: 1300 ~ 1700 ~ 2300 & front: 500 ~ 2500}  

In [ ]:
def cmdd():
    l = struct.calcsize('<hf')
    while True:
        try: 
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            ctrl_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            ctrl_ss.settimeout(30)
            print('Try to connect the server') 
            ctrl_ss.connect((server_ip, ctrl_port))
            print('Succeed a connection')
            while True: 
                code, value = struct.unpack('<hf', ctrl_ss.recv(l)) 
                #print(code, value)
                if   code == 1: kit.motor3.throttle = value
                elif code == 2: kit.motor4.throttle = value
                elif code == 3: pi.set_servo_pulsewidth(rotation_pin, value) 
                elif code == 4: pi.set_servo_pulsewidth(elevation_pin, value) 
        except (socket.timeout, struct.error) as e:
            ctrl_ss.shutdown(socket.SHUT_RDWR) 
            ctrl_ss.close()
            print('Failed to receive any data from server so that control channel closes and will reopen in 5 seconds', e)
            time.sleep(5)      
        except ConnectionRefusedError:
            print('Failed to connect the server and retry in 5 seconds')
            time.sleep(5)
threading.Thread(target=cmdd).start()

In [ ]:
pi.set_servo_pulsewidth(rotation_pin, 0) 
pi.set_servo_pulsewidth(elevation_pin, 0) 

In [ ]:
class SENSOR(): 
    def __init__(self):
        self.data = {}
        self.Temperature = 0 
        self.interval = 3
        nano = serial.Serial(port='/dev/ttyUSB0', baudrate=115200, timeout=3) # serial commu with gps chip
        threading.Thread(target=self.reporter).start()
        time.sleep(3) # wait for the readiness of sensors 
    
    def reporter(self): 
        while True:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sensor_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            sensor_ss.connect((server_ip, sensor_port))
            while True:
                data = {}
                while True:
                    data['P'] = GPRMC.__next__()
                    break
                data = nano.readline()
                try:
                    sensor_ss.send(struct.pack('<H', len(data)))
                    sensor_ss.send(j.encode()) 
                except Exception as e:
                    sensor_ss.shutdown(socket.SHUT_RDWR) 
                    sensor_ss.close()
                    print('sensor channel closes and will reopen in 15 seconds', e)
                    time.sleep(15)
                    break   
                time.sleep(self.interval)    

sensor = SENSOR() 

In [ ]:
camera = picamera.PiCamera(resolution=(640, 480)) 
camera.rotation = 270
camera.start_preview() 
time.sleep(2)

# https://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream
# https://picamera.readthedocs.io/en/release-1.13/recipes2.html#web-streaming

def imgd():
    while True:
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            img_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            img_ss.connect((server_ip, img_port)) 
            img_io = img_ss.makefile('wb')
            stream = io.BytesIO()
            for foo in camera.capture_continuous(stream, 'jpeg'):  
                #camera.annotate_text = f'Celsius:{sensor.Temperature.rstrip()} {dt}'  
                img_io.write(struct.pack('<L', stream.tell()))
                img_io.flush()
                stream.seek(0)
                img_io.write(stream.read()) 
                img_io.flush()
                stream.seek(0)
                stream.truncate()
                time.sleep(1)
        except Exception as e:
            print(e)
            time.sleep(15)
    
threading.Thread(target=imgd).start() 